In [1]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings("ignore")

# pip install pycaret[models] --user

import pycaret
from pycaret.regression import *
from pycaret.regression import predict_model

path = r"D:\OneDrive - NITT\Custom_Download\subset_46.xlsx"
df = pd.read_excel(path)

In [2]:
df.columns = ['opening _jaw','roll_10','closing_jaw','roll_12',
'articulation13','articulation14','epoch_usec','roll_9_360','articulation_degree',
 'open_close'
]

In [3]:
df[(df['roll_9_360']!=0) & (df['articulation_degree']!=0)].head()

,opening _jaw,roll_10,closing_jaw,roll_12,articulation13,articulation14,epoch_usec,roll_9_360,articulation_degree,open_close
1716,0.2118,0.0608,-0.1425,-0.1114,-0.0671,0.0296,1698350427599819,-0.0003,0.3528,90.0
1717,0.2119,0.0609,-0.1425,-0.1100,-0.1016,0.0144,1698350427699853,-0.0003,1.6928,90.0
1718,0.2125,0.0583,-0.1420,-0.1029,-0.0990,0.0401,1698350427799961,-0.0003,4.0328,90.0
1719,0.2134,0.0560,-0.1413,-0.0957,-0.1068,0.0359,1698350427899968,-0.0003,7.0200,90.0
1720,0.2137,0.0506,-0.1403,-0.0854,-0.1073,0.0297,1698350427999885,-0.0003,10.0200,90.0


In [4]:
df.drop(columns='epoch_usec',inplace=True)

In [5]:
def get_best_model(X,target):
    df_temp = X.copy()
    df_temp['target'] = target

    s = setup(df_temp, target = 'target', session_id = 42, fold=5,n_jobs=-1,verbose=False)
    
    best = compare_models(verbose=1)
    return best

# how does articulation angle effect the amount of roll true

In [6]:
torq_12 = df['roll_12']
torq_10 = df['roll_10']
roll_360 = df['roll_9_360']
X = df[['articulation_degree']] 

In [7]:
BestModel_torq_10 = get_best_model(X,torq_10)
BestModel_torq_12 = get_best_model(X,torq_12)
BestModel_roll_360 = get_best_model(X,roll_360)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.0343,0.0028,0.0526,-0.0060,0.0465,1.8561,0.0580
llar,Lasso Least Angle Regression,0.0344,0.0028,0.0527,-0.0090,0.0455,1.8259,0.0160
en,Elastic Net,0.0344,0.0028,0.0527,-0.0090,0.0455,1.8259,0.0160
dummy,Dummy Regressor,0.0344,0.0028,0.0527,-0.0090,0.0455,1.8259,0.0140
lasso,Lasso Regression,0.0344,0.0028,0.0527,-0.0090,0.0455,1.8259,1.1600
lr,Linear Regression,0.0343,0.0028,0.0527,-0.0094,0.0453,1.8282,1.5760
ridge,Ridge Regression,0.0343,0.0028,0.0527,-0.0094,0.0453,1.8282,0.0140
lar,Least Angle Regression,0.0343,0.0028,0.0527,-0.0094,0.0453,1.8282,0.0140
omp,Orthogonal Matching Pursuit,0.0343,0.0028,0.0527,-0.0094,0.0453,1.8282,0.0140
br,Bayesian Ridge,0.0343,0.0028,0.0527,-0.0094,0.0453,1.8282,0.0180


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.0616,0.0068,0.0823,-0.0063,0.0685,2.1176,0.0580
gbr,Gradient Boosting Regressor,0.0619,0.0068,0.0825,-0.0097,0.0687,2.1470,0.0400
catboost,CatBoost Regressor,0.0621,0.0068,0.0826,-0.0115,0.0688,2.1480,0.8820
llar,Lasso Least Angle Regression,0.0620,0.0068,0.0826,-0.0134,0.0666,2.0097,0.0160
dummy,Dummy Regressor,0.0620,0.0068,0.0826,-0.0134,0.0666,2.0097,0.0140
en,Elastic Net,0.0620,0.0068,0.0826,-0.0134,0.0666,2.0097,0.0140
lasso,Lasso Regression,0.0620,0.0068,0.0826,-0.0134,0.0666,2.0097,0.0140
omp,Orthogonal Matching Pursuit,0.0620,0.0068,0.0827,-0.0138,0.0664,2.0061,0.0140
rf,Random Forest Regressor,0.0623,0.0068,0.0826,-0.0138,0.0692,2.1531,0.0740
br,Bayesian Ridge,0.0620,0.0068,0.0827,-0.0138,0.0664,2.0061,0.0140


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,66.8602,16906.7323,129.7599,-0.0048,3.0035,0.9997,0.0220
dummy,Dummy Regressor,67.7994,16950.1570,129.9269,-0.0074,2.7821,299.2606,0.0120
br,Bayesian Ridge,68.0239,16960.9146,129.9722,-0.0082,2.6483,634.9685,0.0140
en,Elastic Net,68.5151,16985.2621,130.0703,-0.0098,2.4560,1076.6227,0.0140
lar,Least Angle Regression,68.5199,16985.5982,130.0717,-0.0098,2.4554,1080.6960,0.0140
omp,Orthogonal Matching Pursuit,68.5199,16985.5982,130.0717,-0.0098,2.4554,1080.6960,0.0140
ridge,Ridge Regression,68.5198,16985.5982,130.0717,-0.0098,2.4554,1080.6953,0.0140
lasso,Lasso Regression,68.5114,16984.9928,130.0693,-0.0098,2.4564,1073.7279,0.0140
lr,Linear Regression,68.5199,16985.5982,130.0717,-0.0098,2.4554,1080.6967,0.0140
llar,Lasso Least Angle Regression,68.5114,16984.9932,130.0693,-0.0098,2.4564,1073.7274,0.0140


In [8]:
def get_prediction(art_degree): 
    test = pd.DataFrame([art_degree],columns=['articulation_degree'])
    
    pred_torq_10 = predict_model(BestModel_torq_10,data=test)['prediction_label'][0]

    pred_torq_12 = predict_model(BestModel_torq_12,data=test)['prediction_label'][0]

    pred_roll_360 = predict_model(BestModel_roll_360,data=test)['prediction_label'][0]

    print(f'Following is needed for articulation_degree of {art_degree}:')
    print(f'torq_10: {pred_torq_10}')
    print(f'torq_12: {pred_torq_12}')
    print(f'roll_360: {pred_roll_360}')

In [9]:
get_prediction(1)

Following is needed for articulation_degree of 1:
torq_10: 0.037846872256800876
torq_12: -0.07252529922716065
roll_360: -2.640082064427238e-05


In [10]:
get_prediction(-45)

Following is needed for articulation_degree of -45:
torq_10: 0.0036876916138040613
torq_12: -0.014635831209983316
roll_360: 2.333481887915305e-05


# how much more torque would it take to roll for the given articulation angle ?

In [11]:
get_prediction(-40)

Following is needed for articulation_degree of -40:
torq_10: 0.011246432963365739
torq_12: -0.04460392363771957
roll_360: 1.7928771104867674e-05


In [12]:
get_prediction(-41)

Following is needed for articulation_degree of -41:
torq_10: 0.011246432963365739
torq_12: -0.04460392363771957
roll_360: 1.9009980659724755e-05
